In [14]:
# run test-mfcc-kmean.ipynb first to get the data

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import hubert, bert, fastspeech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
from malaya_speech.train.model.conformer.model import Model as ConformerModel

class Encoder:
    def __init__(self, config):
        self.config = config
        self.encoder = ConformerModel(**self.config)

    def __call__(self, x, input_mask, training = True):
        return self.encoder(x)
    
config_conformer = malaya_speech.config.conformer_base_encoder_config
config_conformer['subsampling']['type'] = 'none'
config_conformer['dropout'] = 0.0
encoder = Encoder(config_conformer)

In [6]:
cfg = hubert.HuBERTConfig()
model = hubert.Model(cfg, encoder, ['pad', 'eos', 'unk'] + [str(i) for i in range(100)])

In [7]:
X = tf.placeholder(tf.float32, (None, None))
Y = tf.placeholder(tf.int32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [8]:
r = model(X, padding_mask = X_len, target_list = Y)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
dim is deprecated, use axis instead


{'logit_m_list': <tf.Tensor 'hubert/transpose:0' shape=(?, 104) dtype=float32>,
 'logit_u_list': <tf.Tensor 'hubert/transpose_1:0' shape=(?, 104) dtype=float32>,
 'padding_mask': <tf.Tensor 'hubert/LogicalNot:0' shape=(?, ?) dtype=bool>,
 'features_pen': <tf.Tensor 'hubert/Mean:0' shape=() dtype=float32>}

In [9]:
target_m = tf.zeros((tf.shape(r['logit_m_list'])[0],),dtype=tf.int32)
target_u = tf.zeros((tf.shape(r['logit_u_list'])[0],),dtype=tf.int32)

In [10]:
sample_size = tf.cast(tf.shape(target_m)[0], tf.float32)
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target_m, logits = r['logit_m_list'])
entropy_m = tf.reduce_sum(entropy) / sample_size

In [11]:
sample_size = tf.cast(tf.shape(target_u)[0], tf.float32)
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target_u, logits = r['logit_u_list'])
entropy_u = tf.reduce_sum(entropy) / sample_size

In [12]:
loss = entropy_m * 0.95 + entropy_u * 0.05

In [13]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
feat = np.load('out.npy', mmap_mode="r")
leng_path = 'out.len'
with open(leng_path, "r") as f:
    lengs = [int(line.rstrip()) for line in f]
    offsets = [0] + np.cumsum(lengs[:-1]).tolist()
feat.shape

(456, 90)

In [17]:
kmean = hubert.kmeans.ApplyKmeans_TF('kmean.km')

In [18]:
xs, ys = [], []
for offset, leng in zip(offsets, lengs):
    x = feat[offset: offset + leng]
    y = kmean(x) + 3
    ys.append(y)
    xs.append(x)
len(ys), len(xs)

(2, 2)

In [19]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [20]:
X_ = malaya_speech.padding.sequence_1d([y, y1])
X_.shape

(2, 90090)

In [21]:
Y_ = malaya_speech.padding.sequence_1d(ys)
Y_.shape

(2, 281)

In [22]:
o = sess.run([r, loss, optimizer], feed_dict = {X: X_, X_len: [len(y), len(y1)], Y: Y_})

In [23]:
o[1:]

[4.6806192, None]